In [258]:
import pandas as pd
from bs4 import BeautifulSoup
import cv2
import os
import zipfile
import numpy as np
import tensorflow as tf
from inference_sdk import InferenceHTTPClient

In [259]:
def extract_OHRC(file_path, main_path):
    
    new_dir_name = 'final_image'
    extracted_dir = os.path.join(main_path, new_dir_name)
    os.makedirs(extracted_dir, exist_ok=True)
    
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)

In [260]:
def get_image_path(main_path):

    path = r'final_image\browse\calibrated'
    path = os.path.join(main_path, path)
    image = os.listdir(path)
    path = os.path.join(path, image[0])
    image = os.listdir(path)
    
    return os.path.join(path, image[0])

In [261]:
def get_coordinates_change_path(main_path):

    path = r'final_image\geometry\calibrated'
    path = os.path.join(main_path, path)
    coordinates = os.listdir(path)
    path = os.path.join(path, coordinates[0])
    coordinates = os.listdir(path)
    
    return os.path.join(path, coordinates[0])

In [262]:
def get_coordinates_path(main_path):

    path = r'final_image\data\calibrated'
    path = os.path.join(main_path, path)
    coordinates_change = os.listdir(path)
    path = os.path.join(path, coordinates_change[0])
    coordinates_change = os.listdir(path)

    return os.path.join(path, coordinates_change[1])

In [263]:
def get_image_coordinates(coordinates_path):

    with open(coordinates_path, 'r') as file:
        content = file.read()

    soup = BeautifulSoup(content, 'xml')
    soup2 = soup.find('Mission_Area')
    soup_coordinates = soup2.find('isda:Refined_Corner_Coordinates')

    upper_left_latitude = float(soup_coordinates.find('upper_left_latitude').text)
    upper_left_longitude = float(soup_coordinates.find('upper_left_longitude').text)

    upper_right_latitude = float(soup_coordinates.find('upper_right_latitude').text)
    upper_right_longitude = float(soup_coordinates.find('upper_right_longitude').text)

    lower_left_latitude = float(soup_coordinates.find('lower_left_latitude').text)
    lower_left_longitude = float(soup_coordinates.find('lower_left_longitude').text)

    lower_right_latitude = float(soup_coordinates.find('lower_right_latitude').text)
    lower_right_longitude = float(soup_coordinates.find('lower_right_longitude').text)

    upper_left_coordinates = (upper_left_latitude, upper_left_longitude)
    upper_right_coordinates = (upper_right_latitude, upper_right_longitude)
    lower_left_coordinates = (lower_left_latitude, lower_left_longitude)
    lower_right_coordinates = (lower_right_latitude, lower_right_longitude)
    
    return upper_left_coordinates, upper_right_coordinates, lower_left_coordinates, lower_right_coordinates


In [264]:
def get_area_under_observation(image_upper_left_coordinates, image_upper_right_coordinates, image_lower_left_coordinates, image_lower_right_coordinates):
    print(image_upper_left_coordinates, "------------", image_upper_right_coordinates)
    print("           |     ", "          ", "                   |     ")
    print("           |     ", "          ", "                   |     ")
    print("           |     ", "          ", "                   |     ")
    print("           |     ", "          ", "                   |     ")
    print(image_lower_left_coordinates, "------------", image_lower_right_coordinates)

In [265]:
def get_image_dimentions(image_path):
    
    OHRC_image = cv2.imread(image_path)
    height , width = OHRC_image.shape[:2]

    return height, width

In [266]:
def creat_data_frame(coordinates_change_path):

    df = pd.read_csv(coordinates_change_path)
    df['Pixel'] = df['Pixel']//10
    df['Scan'] = df['Scan']//10

    return df

In [267]:
def creat_image(image_path):

    image = cv2.imread(image_path)

    return image

In [268]:
def get_big_images(OHRC_image, height, width):

    new_dir_name = 'big_images'
    big_images = os.path.join(main_path, new_dir_name)
    os.makedirs(big_images, exist_ok=True)

    n = height//width
    big_images_number = 0
    
    for i in range(n):
        temp_image = image[i*width : (i+1)*width , 0: width]
        big_image_path = os.path.join(big_images, f'patch_{big_images_number}.jpg')
        cv2.imwrite(big_image_path, temp_image)
        big_images_number += 1
        
    last_image = image[ n*width : height, 0:width]
    big_image_path = os.path.join(big_images, f'patch_{big_images_number}.jpg')
    cv2.imwrite(big_image_path, last_image)

    print(f'images store in folder "big_image" in path : {big_images}')

    return big_images

In [269]:
def get_coordinates_from_csv(x, y, data_frame):
    pixel = y - (y % 10)
    scan = x - (x % 10)

    row = data_frame[(data_frame['Pixel'] == pixel) & (data_frame['Scan'] == scan)]
    
    if not row.empty:
        longitude, latitude = row[['Longitude', 'Latitude']].values[0]
        print(longitude, latitude)
    else:
        longitude, latitude = None, None
        print("No matching coordinates found.")

    return float(latitude), float(longitude)


In [312]:
import base64

def read_and_encode_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_image

def get_coordinates(big_images_path, main_path, images, window_size, width, height, data_frame):

    crater_data = {'carter_pi_co' : [] , 'size' : [], 'top_left' : [], 'bottom_right' : [], 'coordintes' : []}
    boulder_data = {'boulder_pi_co' : [], 'size' : [], 'top_left' : [], 'bottom_right' : [], 'coordintes' : []}

    OHRC_image = images 
    n = window_size // width
    big_images = os.listdir(big_images_path)
    
    for img, i  in zip(big_images, range(len(big_images))):
        img_path = os.path.join(big_images_path, img)

        CLIENT = InferenceHTTPClient(
            api_url="https://detect.roboflow.com",
            api_key="W4SnHSPYMpGk4nhQVPUA"
        )

        encoded_image = read_and_encode_image(img_path)
        prediction = CLIENT.infer(encoded_image, model_id="crater-and-boulder-detection/2")

        for data in prediction['predictions']:

            x1, y1 = data['x'], data['y']
            x1, y1 = normalize_for_OHRC_image(x1, y1, i, width)
            x_size, y_size = data['width'], data['height']
            row = df[(df['Pixel'] == int(x1)-int(x1)%10 ) & (df['Scan'] == int(y1)-int(y1)%10 )]
            latitude, longitude = row['Longitude'].values[0], row['Latitude'].values[0]

            if data['class'] == 'crater':
                crater_data['carter_pi_co'].append((x1, y1))
                crater_data['size'].append((x_size, y_size))
                crater_data['top_left'].append((x1-(x_size)/2, y1-(y_size)/2))
                crater_data['bottom_right'].append((x1+(x_size)/2, y1+(y_size)/2))
                crater_data['coordintes'].append((latitude, longitude))


            else:
                boulder_data['boulder_pi_co'].append((x1, y1))
                boulder_data['size'].append((x_size, y_size))
                boulder_data['top_left'].append((x1-(x_size)/2, y1-(y_size)/2))
                boulder_data['bottom_right'].append((x1+(x_size)/2, y1+(y_size)/2))
                boulder_data['coordintes'].append((latitude, longitude))

    crater_data = pd.DataFrame(crater_data)
    boulder_data = pd.DataFrame(boulder_data)

    return crater_data, boulder_data


In [271]:
def normalize_for_big_image(x1, y1, x2, y2, x, y, window_size):

    x1 += window_size * x
    x2 += window_size * x
    y1 += window_size * y 
    y2 += window_size * y

    return x1, x2, y1, y2

In [272]:
def normalize_for_OHRC_image(x1, y1, big_image_no, width):

    y1 = y1 + (width * big_image_no)

    return x1, y1

In [273]:
def find_crater_boulder_in_radius(dataframe):
    coordinates = (0, 0)

    coordinates[0] = float(input("enter latitude : "))
    coordinates[1] = float(input("enter longitude : "))
    radius = float(input("enter radius : "))

    pi_radius = 1200/3000 * radius

    row = dataframe[(dataframe['Latitude'] == coordinates[0]) & (dataframe['Longitude'] == coordinates[1])]

    if not row.empty:
        pi_x , pi_y = row[['Pixel', 'Scan']].values[0]
    else:
        pi_x, pi_y  = None, None
        print("coordinates not in image.")

    data = dataframe[(dataframe['Pixel'] > pi_x - pi_radius) & (dataframe['Pixel'] < pi_x + pi_radius) &
                    (dataframe['Scan'] > pi_y - pi_radius) & (dataframe['scan'] < pi_x + pi_radius)]

    data['Pixel'] = data['Pixel'].apply(lambda x : abs(x - pi_x))
    data['Scan'] = data['Scan'].apply(lambda x : abs(x - pi_y))

    data = data[(data['Pixel'])**2 + (data['Scan'])**2 <= r**2]

    return data


In [343]:
def draw_bounding_box(image_path, crater_df, boulder_df, main_path):
    OHRC_image = cv2.imread(image_path)

    crater_df['top_left'] = crater_df['top_left'].apply(lambda x: (int(x[0]), int(x[1])))
    crater_df['bottom_right'] = crater_df['bottom_right'].apply(lambda x: (int(x[0]), int(x[1])))

    boulder_df['top_left'] = crater_df['top_left'].apply(lambda x: (int(x[0]), int(x[1])))
    boulder_df['bottom_right'] = crater_df['bottom_right'].apply(lambda x: (int(x[0]), int(x[1])))

    for pt1, pt2, text in zip(crater_df['top_left'], crater_df['bottom_right'], crater_df['coordintes']):
        
        cv2.rectangle(OHRC_image, pt1, pt2, (0, 255, 0), 7)
        cv2.putText(OHRC_image, str(text), (pt1[0], pt1[1] - 20) , cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 255, 100), 2, cv2.LINE_AA)

    for pt1, pt2 in zip(boulder_df['top_left'], boulder_df['bottom_right']):
        
        cv2.rectangle(OHRC_image, pt1, pt2, (0, 0, 255), 7)
        cv2.putText(OHRC_image, str(text), (pt1[0], pt1[1] - 20) , cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 100, 255), 2, cv2.LINE_AA)

    new_image_path = os.path.join(main_path, 'new_image.jpg')
    cv2.imwrite(new_image_path, OHRC_image)
    
    return new_image_path

In [275]:
        ## main ##

In [276]:
file_path = 'C:\Hackathon\crater_boulder_detection\ch2_ohr_ncp_20210405T1606536730_d_img_d18.zip'
main_path = r'C:\Hackathon\crater_boulder_detection'

extract_OHRC(file_path, main_path)

In [277]:
image_path = get_image_path(main_path)
coordinates_path = get_coordinates_path(main_path)
coordinates_change_path = get_coordinates_change_path(main_path)

In [278]:
image_upper_left_coordinates, image_upper_right_coordinates, image_lower_left_coordinates, image_lower_right_coordinates = get_image_coordinates(coordinates_path)
get_area_under_observation(image_upper_left_coordinates,image_upper_right_coordinates, image_lower_left_coordinates, image_lower_right_coordinates)

(-2.576048, 336.486234) ------------ (-2.579083, 336.589455)
           |                                    |     
           |                                    |     
           |                                    |     
           |                                    |     
(-3.413866, 336.484646) ------------ (-3.416904, 336.587773)


In [279]:
height, width = get_image_dimentions(image_path)

In [280]:
df = creat_data_frame(coordinates_change_path)
image = creat_image(image_path)

In [281]:
big_images_path = get_big_images(image, height, width)

images store in folder "big_image" in path : C:\Hackathon\crater_boulder_detection\big_images


In [320]:
crater_data, boulder_data = get_coordinates(big_images_path, main_path, image, 600, width, height, df)

In [323]:
crater_data

,carter_pi_co,size,top_left,bottom_right,coordintes
0,"(315.0, 704.5)","(40.0, 49.0)","(295.0, 680.0)","(335.0, 729.0)","(336.51286, -2.63943327)"
1,"(477.0, 713.0)","(44.0, 46.0)","(455.0, 690.0)","(499.0, 736.0)","(336.526639, -2.64073373)"
2,"(726.0, 250.5)","(32.0, 37.0)","(710.0, 232.0)","(742.0, 269.0)","(336.54822, -2.60023294)"
3,"(388.0, 282.5)","(34.0, 37.0)","(371.0, 264.0)","(405.0, 301.0)","(336.518961, -2.60205316)"
4,"(799.5, 306.0)","(27.0, 32.0)","(786.0, 290.0)","(813.0, 322.0)","(336.554229, -2.60488007)"
...,...,...,...,...,...
266,"(918.0, 8762.5)","(38.0, 35.0)","(899.0, 8745.0)","(937.0, 8780.0)","(336.563039, -3.36169631)"
267,"(248.5, 9332.0)","(49.0, 34.0)","(224.0, 9315.0)","(273.0, 9349.0)","(336.505343, -3.41097527)"
268,"(39.0, 8431.0)","(44.0, 46.0)","(17.0, 8408.0)","(61.0, 8454.0)","(336.487397, -3.32994528)"
269,"(598.5, 8948.0)","(47.0, 40.0)","(575.0, 8928.0)","(622.0, 8968.0)","(336.535528, -3.37698842)"


In [324]:
boulder_data

,boulder_pi_co,size,top_left,bottom_right,coordintes
0,"(482.0, 823.5)","(32.0, 33.0)","(466.0, 807.0)","(498.0, 840.0)","(336.52748, -2.65059865)"
1,"(987.0, 636.0)","(36.0, 42.0)","(969.0, 615.0)","(1005.0, 657.0)","(336.570486, -2.63487143)"
2,"(1150.0, 516.0)","(40.0, 42.0)","(1130.0, 495.0)","(1170.0, 537.0)","(336.58509, -2.6245651)"
3,"(754.5, 754.5)","(33.0, 35.0)","(738.0, 737.0)","(771.0, 772.0)","(336.550713, -2.64502281)"
4,"(1109.0, 838.0)","(38.0, 36.0)","(1090.0, 820.0)","(1128.0, 856.0)","(336.580746, -2.65305273)"
...,...,...,...,...,...
94,"(213.5, 8725.5)","(45.0, 27.0)","(191.0, 8712.0)","(236.0, 8739.0)","(336.502862, -3.35634476)"
95,"(1078.5, 8462.5)","(35.0, 21.0)","(1061.0, 8452.0)","(1096.0, 8473.0)","(336.576807, -3.33526428)"
96,"(992.5, 8660.5)","(31.0, 25.0)","(977.0, 8648.0)","(1008.0, 8673.0)","(336.569912, -3.35295243)"
97,"(136.0, 9034.0)","(40.0, 38.0)","(116.0, 9015.0)","(156.0, 9053.0)","(336.495914, -3.38386331)"


In [344]:
draw_bounding_box(image_path, crater_data, boulder_data, main_path)

'C:\\Hackathon\\crater_boulder_detection\\new_image.jpg'

In [330]:
df

,Longitude,Latitude,Pixel,Scan
0,336.486234,-2.576048,0,0
1,336.487097,-2.576074,10,0
2,336.487961,-2.576099,20,0
3,336.488824,-2.576125,30,0
4,336.489687,-2.576150,40,0
...,...,...,...,...
113493,336.584356,-3.416805,1160,9369
113494,336.585212,-3.416830,1170,9369
113495,336.586069,-3.416855,1180,9369
113496,336.586925,-3.416880,1190,9369
